In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

AssertionError: Duplicate registrations for type 'optimizer'

In [ ]:
# для google colab
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# розмір картинки
IMG_SIZE = (75, 75)

# папка з датасетом у якій підпапки з фотографіями кожного символа 
# назви класів будуть відповідати іменам підпапок
directory = "images/"

In [ ]:
data_generator = ImageDataGenerator(validation_split = 0.2, rescale = 1.0 / 255)
 
# датасет для тренування
train = data_generator.flow_from_directory(directory,
                                               target_size = IMG_SIZE,
                                               class_mode = 'categorical',
                                               subset = 'training')

# датасет для тестування
test = data_generator.flow_from_directory(directory,
                                               target_size = IMG_SIZE,
                                               class_mode = 'categorical',
                                               subset = 'validation')

In [ ]:
def createModel(image_shape = IMG_SIZE):
    '''
    Аргументи:
        image_shape -- ширина та висота картинки
    Повертає:
        tf.keras.model
    '''
    input_shape = image_shape + (3,)
    
    base_model = InceptionV3(input_shape = input_shape,
                            include_top = False,
                            weights = "imagenet",
                            classes = 29,
                            classifier_activation = "softmax")
    # заморозимо шари, щоб їх не тренувати
    base_model.trainable = False

    # вхідний шар
    inputs = tf.keras.Input(shape = input_shape) 

    # для нормалізації інпута [-1;1]
    x = tf.keras.applications.inception_v3.preprocess_input(inputs) 

    x = base_model(x, training = False) 
    x = tfl.GlobalAveragePooling2D()(x) 
    x = tfl.Dropout(0.2)(x)
    outputs = tfl.Dense(29, activation = "softmax")(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
model = createModel(IMG_SIZE)

In [ ]:
# метрики
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision_f1 = precision(y_true, y_pred)
    recall_f1 = recall(y_true, y_pred)
    return 2 * ((precision_f1 * recall_f1) / (precision_f1 + recall_f1 + K.epsilon()))

In [ ]:
base_learning_rate = 0.01

initial_epochs = 10

loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam(base_learning_rate)
metrics = ['acc', f1_score, precision, recall]

In [ ]:
model.compile(optimizer = optimizer,
              loss = loss_function,
              metrics = metrics)

In [ ]:
history = model.fit(train,
                    validation_data = test,
                    epochs = initial_epochs)

In [ ]:
# model.summary()

# print("Кількість шарів у базовій моделі: ", len(tunedModel.layers))

In [ ]:
tunedModel = model.layers[3]
tunedModel.trainable = True

fine_tune_start = 250

In [ ]:
# Заморозимо всі шари перед fine_tune_start
for layer in tunedModel.layers[:fine_tune_start]:
    layer.trainable = False

optimizer = Adam(learning_rate = 0.0001 * base_learning_rate)

fine_tune_epochs = 100
total_epochs =  initial_epochs + fine_tune_epochs

In [ ]:
model.compile(optimizer = optimizer,
              loss = loss_function,
              metrics = metrics)

In [ ]:
history_fine = model.fit(train,
                         validation_data = test,
                         epochs = total_epochs,
                         initial_epoch = history.epoch[-1])

In [ ]:
model.save('Fine_tuned_model01.h5', overwrite = True, include_optimizer = True)